<a href="https://colab.research.google.com/github/sahiliem/GPT2_Text_Summarization/blob/main/GPT2_for_text_sumarization1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Let's Install the Requirements

In [ ]:
!pip install layer --upgrade -q
!pip install sentencepiece -q
!pip install transformers -q

     |████████████████████████████████| 136 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 57.0 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
     |████████████████████████████████| 381 kB 63.9 MB/s 
     |████████████████████████████████| 231 kB 55.9 MB/s 
     |████████████████████████████████| 256 kB 57.2 MB/s 
     |████████████████████████████████| 4.0 MB 60.5 MB/s 
     |████████████████████████████████| 2.4 MB 53.7 MB/s 
     |████████████████████████████████| 4.4 MB 49.5 MB/s 
     |████████████████████████████████| 26.7 MB 1.2 MB/s 
     |████████████████████████████████| 17.8 MB 413 kB/s 
     |████████████████████████████████| 271 kB 47.1 MB/s 
     |████████████████████████████████| 4.2 MB 48.1 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 3.1 MB 49.6 MB/s 
     |████████████████████████████████| 147 kB 60.4 MB/s 
     |████████████████████████████████| 192 kB 59.7 MB/s 
     |███████████

In [ ]:
from layer.decorators import dataset, model,resources, pip_requirements, fabric
from layer import Model
import layer
import torch
import random
import re
import pandas as pd
import numpy as np
import os

## Getting started with Layer:
- Login to Layer Console
- Initialize the project

In [ ]:
# Login to Layer:
layer.login()

In [ ]:
# Initialize the project:
layer.init('GPT2_text_summarization')

Project(name='GPT2_text_summarization', raw_datasets=[], derived_datasets=[], models=[], path=PosixPath('.'), project_files_hash='', readme='', account=Account(id=UUID('c9442765-3fe4-4729-a7e4-d3fc9b07726b'), name='aymane_hachcham'), _id=UUID('f29c025a-0d92-4913-94ff-035d4de50df7'), functions=[])

## Process and Load the Dataset to Layer

In [ ]:
# Mount Google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the dataset:
data_path = '/content/drive/MyDrive/GPT2/Dataset/Reviews.csv'
import pandas as pd

reviews = pd.read_csv(data_path)

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [ ]:
reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Data Processing and Cleaning

In [ ]:
# Check for null values:
print(reviews.isna().sum())

# Remove null values:
reviews.dropna(inplace=True)

# Combining the two columns review and summary:
reviews['training'] = reviews['Text']  + 'TL;DR' + reviews['Summary']

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64


In [ ]:
reviews[:1500].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 1499
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      1500 non-null   int64 
 1   ProductId               1500 non-null   object
 2   UserId                  1500 non-null   object
 3   ProfileName             1500 non-null   object
 4   HelpfulnessNumerator    1500 non-null   int64 
 5   HelpfulnessDenominator  1500 non-null   int64 
 6   Score                   1500 non-null   int64 
 7   Time                    1500 non-null   int64 
 8   Summary                 1500 non-null   object
 9   Text                    1500 non-null   object
 10  training                1500 non-null   object
dtypes: int64(5), object(6)
memory usage: 140.6+ KB


In [ ]:
# We take 5000 rows from the whole data. So can avoid GPU out of memory:
reviews = reviews[['Summary','Text','training']][:5000]

In [ ]:
# Save the dataset to Layer:
@dataset('amazon_reviews')
@resources(data_path)
def read_reviews():
    df = pd.read_csv(data_path) 
    # Remove null values:
    df.dropna(inplace=True)  
    # Combining the two columns review and summary:
    df['training'] = df['Text'] + 'TL;DR' + df['Summary'] 
    df = df[['Summary','Text','training']][:5000]
    return df

read_reviews()

Output()

,Summary,Text,training
0,Good Quality Dog Food,I have bought several of the Vitality canned d...,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Product arrived labeled as Jumbo Salted Peanut...
2,"""Delight"" says it all",This is a confection that has been around a fe...,This is a confection that has been around a fe...
3,Cough Medicine,If you are looking for the secret ingredient i...,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...,Great taffy at a great price. There was a wid...
...,...,...,...
4995,The cavemen must have been wealthy,I really wanted to like these.<br /><br />Firs...,I really wanted to like these.<br /><br />Firs...
4996,These cookies need work; you can make your own...,I was not impressed with these cookies when I ...,I was not impressed with these cookies when I ...
4997,Okay in a pinch - not great,The cookies came sealed and seem to be high qu...,The cookies came sealed and seem to be high qu...
4998,"they are good (except for the ""rainforest"" fla...","These taste very good, but aren't like the BES...","These taste very good, but aren't like the BES..."


In [ ]:
# Take the average length of the reviews:

sum_all_tokens = sum([len(review.split()) for review in reviews['training']])
avg_length = sum_all_tokens / len(reviews['training'])
avg_length

78.1608

Since the average instance length in words is 80, we can assume that a max length of 150 will cover most of the instances.

In [ ]:
max_length = 100

In [ ]:
reviews['training']

0       I have bought several of the Vitality canned d...
1       Product arrived labeled as Jumbo Salted Peanut...
2       This is a confection that has been around a fe...
3       If you are looking for the secret ingredient i...
4       Great taffy at a great price.  There was a wid...
                              ...                        
4995    I really wanted to like these.<br /><br />Firs...
4996    I was not impressed with these cookies when I ...
4997    The cookies came sealed and seem to be high qu...
4998    These taste very good, but aren't like the BES...
4999    I love these cookies. I am on the paleo diet r...
Name: training, Length: 5000, dtype: object

## Load the Model and the Tokenizer from Layer

In [ ]:
@model("gpt2")
@fabric("f-medium")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_tokenizer():
    from transformers import AutoTokenizer
    # Load tokenizer from Hugging face
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    return tokenizer

@model("gpt2")
@pip_requirements(packages=["torch","transformers","sentencepiece"])
def build_model():
    from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
    from torch import cuda
    import torch
    from torch.utils.data import Dataset, DataLoader
    from transformers import AutoModelWithLMHead
    import torch.optim as optim
    
    parameters={
        "BATCH_SIZE":32,          
        "EPOCHS":10,              
        "LEARNING_RATE":3e-4,          
        "MAX_SOURCE_TEXT_LENGTH":150,   
        "MAX_TARGET_TEXT_LENGTH":150
    }

    # Log parameters to Layer
    layer.log(parameters)
  

    # Load pretrained model from Hugging face
    model = AutoModelWithLMHead.from_pretrained("gpt2")
    device = 'cuda' if cuda.is_available() else 'cpu'
    model.to(device)
    
    return model

In [ ]:
layer.run([build_tokenizer, build_model], debug=True)

Output()

13:07:47 gpt2: Starting setup of dependencies...
13:07:47 gpt2: Using selector: EpollSelector
13:07:47 gpt2: Successfully logged into https://app.layer.ai
13:07:47 gpt2: Using selector: EpollSelector
13:07:47 gpt2: Downloading execution artifacts model-training--layer2022040715233796120000001d/c9442765-3fe4-4729-a7e4-d3fc9b07726b/9e3a58a5-7c08-410d-b005-5bd2435da13a/gpt2-train.tgz to ~/source
13:07:47 gpt2: Creating directory ~/source
13:07:47 gpt2: Place __init__.py in ~/source
13:07:47 gpt2: Download binary(c9442765-3fe4-4729-a7e4-d3fc9b07726b/9e3a58a5-7c08-410d-b005-5bd2435da13a/gpt2-train.tgz) to temp directory
13:07:48 gpt2: Binary archive gpt2-train.tgz downloaded and extracted to ~/source successfully
13:07:48 gpt2: Execution artifacts model-training--layer2022040715233796120000001d/c9442765-3fe4-4729-a7e4-d3fc9b07726b/9e3a58a5-7c08-410d-b005-5bd2435da13a/gpt2-train.tgz downloaded successfully to ~/source
13:07:48 gpt2: Installing python dependencies from ~/source/requirements.t

Run(project_name='GPT2_text_summarization')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

tokenizer = build_tokenizer()
extra_length = len(tokenizer.encode(" TL;DR "))
 
class GPT2ReviewDataset(Dataset):  
    def __init__(self, tokenizer, reviews, max_len):
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.eos = self.tokenizer.eos_token
        self.eos_id = self.tokenizer.eos_token_id
        self.reviews = reviews
        self.result = []

        for review in self.reviews:
            # Encode the text using tokenizer.encode(). We add EOS at the end
            tokenized = self.tokenizer.encode(review + self.eos)
            
            # Padding/truncating the encoded sequence to max_len 
            padded = self.pad_truncate(tokenized)            

            # Creating a tensor and adding to the result
            self.result.append(torch.tensor(padded))

    def __len__(self):
        return len(self.result)


    def __getitem__(self, item):
        return self.result[item]

    def pad_truncate(self, name):
        extra_length = 4
        name_length = len(name) - extra_length
        if name_length < self.max_len:
            difference = self.max_len - name_length
            result = name + [self.eos_id] * difference
        elif name_length > self.max_len:
            result = name[:self.max_len + 3]+[self.eos_id] 
        else:
            result = name
        return result

Output()

In [ ]:
dataset = GPT2ReviewDataset(tokenizer, reviews['training'], max_length)

Token indices sequence length is longer than the specified maximum sequence length for this model (1430 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
for item in dataset:
  print(item)
  break

tensor([   40,   423,  5839,  1811,   286,   262, 28476,   414, 32530,  3290,
         2057,  3186,   290,   423,  1043,   606,   477,   284,   307,   286,
          922,  3081,    13,   383,  1720,  3073,   517,   588,   257, 20798,
          621,   257, 13686,  6174,   290,   340, 25760,  1365,    13,  2011,
        45246,   318,   957, 17479,   290,   673,  5763,   689,   428,  1720,
         1365,   621,   220,   749,    13, 14990,    26,  7707, 10248, 14156,
         8532,  7318, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256])


In [ ]:
# The training Loop:
# @pip_requirements(packages=["transformers","sentencepiece"])
# @fabric("f-gpu-small")
# @model("gpt2")
def train(model, optimizer, dl, epochs):    
    for epoch in range(epochs):
        for idx, batch in enumerate(dl):
             with torch.set_grad_enabled(True):
                optimizer.zero_grad()
                batch = batch.to('cuda')
                output = model(batch, labels=batch)
                loss = output[0]
                loss.backward()
                optimizer.step()
                if idx % 50 == 0:
                    print("loss: %f, %d"%(loss, idx))

In [ ]:
layer.run([train])

ConfigError: ignored

In [ ]:
model = build_model()

Output()

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:911: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
import torch.optim as optim
optimizer = optim.Adam(params = model.parameters(), lr=3e-4)

In [ ]:
train(model=model, optimizer=optimizer, dl=dataloader, epochs=10)

loss: 2.198657, 0
loss: 2.288299, 50


In [ ]:
import torch

def topk(probs, n=9):
    # The scores are initially softmaxed to convert to probabilities
    probs = torch.softmax(probs, dim= -1)
    
    # PyTorch has its own topk method, which we use here
    tokensProb, topIx = torch.topk(probs, k=n)
    
    # The new selection pool (9 choices) is normalized
    tokensProb = tokensProb / torch.sum(tokensProb)

    # Send to CPU for numpy handling
    tokensProb = tokensProb.cpu().detach().numpy()

    # Make a random choice from the pool based on the new prob distribution
    choice = np.random.choice(n, 1, p = tokensProb)
    tokenId = topIx[choice][0]

    return int(tokenId)

In [ ]:
def model_infer(model, tokenizer, review, max_length=15):
    # Preprocess the init token (task designator)
    review_encoded = tokenizer.encode(review)
    result = review_encoded
    initial_input = torch.tensor(review_encoded).unsqueeze(0).to('cuda')

    with torch.set_grad_enabled(False):
        # Feed the init token to the model
        output = model(initial_input)

        # Flatten the logits at the final time step
        logits = output.logits[0,-1]

        # Make a top-k choice and append to the result
        result.append(topk(logits))

        # For max_length times:
        for _ in range(max_length):
            # Feed the current sequence to the model and make a choice
            input = torch.tensor(result).unsqueeze(0).to('cuda')
            output = model(input)
            logits = output.logits[0,-1]
            res_id = topk(logits)

            # If the chosen token is EOS, return the result
            if res_id == tokenizer.eos_token_id:
                return tokenizer.decode(result)
            else: # Append to the sequence 
                result.append(res_id)

    # IF no EOS is generated, return after the max_len
    return tokenizer.decode(result)

In [ ]:
sample_reviews = reviews['training'].sample(n=1, random_state=1)

In [ ]:
review = "My local coffee shop has me addicted to their 20 oz vanilla chai lattes. At $3.90 a pop I was spending a lot of money.  I asked what brand they used, need nutritional information, of course!  They told me it was Big Train Chai Vanilla.<br />It's important to follow the directions on the can.  I made mine with just milk with a yucky result.  Use the water with a little milk as there is milk powder in the mix.<br /><br />WARNING:It's addicting!!!"

In [ ]:
summary = [model_infer(model, tokenizer, review).strip() for review in sample_reviews]

In [ ]:
summary

["No Microwave Directions on Box!TL;DRI look forward to cooking in the microwave...means no spattered stove to clean! BIG SURPRISE - no microwave directions. Well this will be a noble experiment because it will be microwaved! Called Miss Betty Crocker @ General Mills and they don't supply microwaved directions either. So wish me luck as I experiment!<br /><br />UPDATE: Found Microwave directions on web BUT you still have to experiment :(<br /><br />MICROWAVE DIRECTIONS<br />1. Crumble 1 lb lean ground beef into 3-qt microwavable casserole or bowl. Microwave uncovered on High 3 to 5 minutes, breaking up beef after 3 minutes, until brown; drain.<br />2. Stir in sauce mix, 2-2/3 cups boiling water, 1 cup milk and uncooked potatoes.<br />3. Microwave uncovered on High 14 to 21 minutes, stirring every 7 minutes, until potatoes are tender (sauce will thicken as it stands). Dish will be hot. Meanwhile, stir 1/4 cup milk and toping mix in bowl 30 seconds; set aside.<br />4. Pour topping over b

In [ ]:
for review in sample_reviews:
    summaries = set()
    print(review)
    while len(summaries) < 3:
        summary = model_infer(model, tokenizer, review).strip()
        if summary not in summaries:
            summaries.add(summary)
    print("Summaries: "+ str(summaries) +"\n")

I look forward to cooking in the microwave...means no spattered stove to clean! BIG SURPRISE - no microwave directions. Well this will be a noble experiment because it will be microwaved! Called Miss Betty Crocker @ General Mills and they don't supply microwaved directions either. So wish me luck as I experiment!<br /><br />UPDATE: Found Microwave directions on web BUT you still have to experiment :(<br /><br />MICROWAVE DIRECTIONS<br />1. Crumble 1 lb lean ground beef into 3-qt microwavable casserole or bowl. Microwave uncovered on High 3 to 5 minutes, breaking up beef after 3 minutes, until brown; drain.<br />2. Stir in sauce mix, 2-2/3 cups boiling water, 1 cup milk and uncooked potatoes.<br />3. Microwave uncovered on High 14 to 21 minutes, stirring every 7 minutes, until potatoes are tender (sauce will thicken as it stands). Dish will be hot. Meanwhile, stir 1/4 cup milk and toping mix in bowl 30 seconds; set aside.<br />4. Pour topping over beef mixture. (If topping is too thick 

KeyboardInterrupt: ignored